In [ ]:
import os
from os import path
import sys
import pickle
from collections import namedtuple, defaultdict, Counter
from datetime import datetime
from time import time
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats
import cohd_temporal as cohd

In [ ]:
dir_data = '/path/to/data/dir'
file_pcs = path.join(dir_data, 'patient_code_sequences.txt')
file_persons = path.join(dir_data, 'persons.csv')
file_concepts = path.join(dir_data, 'concepts.csv')
file_ingredients = path.join(dir_data, 'drug_ingredients.csv')
file_condition_ancestors = path.join(dir_data, 'condition_ancestors.csv')
file_cads = path.join(dir_data, 'concept_age_dists.pkl')
file_cagds = path.join(dir_data, 'concept_age_group_dists.pkl')
file_deltas = path.join(dir_data, 'deltas.pkl')
file_backup_suffix = '.backup'

## Load data into dataframe

In [ ]:
# column indices for persons.csv
df_persons = pd.read_csv(file_persons, sep='\t', header=0, index_col=0, 
                         parse_dates=['birth_date'], infer_datetime_format=True)

# Check the data types of the columns
df_persons.dtypes

In [ ]:
# Load the concept definitions
df_concepts = pd.read_csv(file_concepts, sep='\t', header=0, index_col='concept_id')
df_concepts.head()

In [ ]:
ancestor_map = defaultdict(list)

# Read the in the mappings from drugs to ingredients
df_ingredients = pd.read_csv(file_ingredients, sep='\t', header=0)
print(df_ingredients.dtypes)

# Create a map from the observed drug_concept_id to the ingredient
for index, row in df_ingredients.iterrows():
    ancestor_map[int(row['drug_concept_id'])].append(int(row['ancestor_concept_id']))
    
# Read in the mappings for conditions
df_condition_ancestors = pd.read_csv(file_condition_ancestors, sep='\t', header=0)
print(df_condition_ancestors.dtypes)

# Add the conditions to the ancestor map
for index, row in df_condition_ancestors.iterrows():
    ancestor_map[int(row['condition_concept_id'])].append(int(row['ancestor_concept_id']))

## Calculate age distributions for each concept

In [ ]:
# Helpers for reading in the patient_code_sequences.txt

Occurrence = namedtuple('Occurrence', ['concept_id', 'date'])


def _process_occurrence_str(x):
    concept_id_str, date_str = x.split(',')
    occ = Occurrence(int(concept_id_str), datetime.strptime(date_str.strip(), '%Y-%m-%d'))
    return occ


def _trim_low_occurrences(occurrences, cads):
    """ Removes occurrences if they're not in cads (i.e., count <= 10) 
    
    Removes occurrences from the list passed in, does not create a new list """
    for i in range(len(occurrences)-1, -1, -1):
        if occurrences[i].concept_id not in cads:
            del occurrences[i]


def _expand_occurrence_ancestors(occurrences, ancestor_map):
    """ Expand a list of occurrences to include the ancestors defined in ancestor_map 
    Only adds the ancestor the first it occurs, i.e., if two concepts have the same ancestor,
    the ancestor is only added the first time """
    occurrences_expanded = list()
    observed_concepts = list()
    for occurrence in occurrences:
        concept_id = occurrence.concept_id
        dt = occurrence.date
        
        # If we've already seen this concept (it must have been an ancestor of a previous concept), 
        # then move onto the next occurrence in the list
        if concept_id in observed_concepts:
            continue
            
        # First time we're seeing this concept, add it to the lists
        occurrences_expanded.append(occurrence)
        observed_concepts.append(concept_id)
        
        # Now add its ancestors
        if concept_id in ancestor_map:
            ancestors = ancestor_map[concept_id]
            for anc in ancestors:
                # Add ancestors that haven't already been observed
                if anc not in observed_concepts:
                    occurrences_expanded.append(Occurrence(anc, dt))
                    observed_concepts.append(anc)
                    
    return occurrences_expanded

In [ ]:
def _get_age(date, dob):
    """ Gets the age of the person (years) on a date """
    # if the date occurs before the date of birth, return 0
    if date < dob:
        return 0
    
    years = date.year - dob.year
    if date.month < dob.month or (date.month == dob.month and date.day < dob.day):
        years -= 1
    return years


In [ ]:
save_intermediates = False

# For keeping track of processing time
t1 = time()

# cads - concept age distributions: holds the age (year) of first occurrence of each concept
# keys are the concept_id (int)
cads = defaultdict(AgeCounter)
# cads = defaultdict(Counter)

count = 0
file_cads_backup = file_cads + file_backup_suffix

# Read patient_code_sequences.txt
with open(file_pcs) as fh:  
    for line in fh:
        split = line.strip().split('\t')
        
        # person_id is the first entry
        pid = int(split.pop(0))
        
        # Get the person's date of birth
        dob = df_persons.loc[pid, 'birth_date']
        
        # Process the remaining string into a list of Occurrences
        occurrences = [_process_occurrence_str(x) for x in split]
        
        # Expand the list of occurrences with the first instance of their ancestors
        occurrences = _expand_occurrence_ancestors(occurrences, ancestor_map)
        n_occurrences = len(occurrences)               
        
        # Track a distribution of ages for each concept
        for occ in occurrences:
            age = _get_age(occ.date, dob)
            concept_id = occ.concept_id
            cads[concept_id].add_age(age)
        
        # Display progress
        count += 1
        if count % 100000 == 0:
            # Processing time and size of data structure
            ellapsed_time = (time() - t1) / 60
            print(f'{count} - {ellapsed_time:.01f} min')
            
            # Save a backup copy of the data
            if save_intermediates:
                with open(file_cads_backup, 'wb') as f:
                    pickle.dump(cads, f)    
                
# Find concept-age distributions with total concept count <= 10 for deletion
concepts_to_remove = list()
for concept, cad in cads.items():
    total_count = np.sum(cad.counts)
    if (total_count) <= 10:
        # Can't delete from the list while iterating, so keep track of concepts to delete
        concepts_to_remove.append(concept)
        
# Delete low-count concepts from cads
for concept in concepts_to_remove:
    del cads[concept]
del concepts_to_remove

# Save the concept age distributions            
with open(file_cads, 'wb') as f:
    pickle.dump(cads, f)

# Delete the backup file
if save_intermediates and path.exists(file_cads_backup):
    os.remove(file_cads_backup)

# Display overall processing time
ellapsed_time = (time() - t1) / 60
print(f'{count} - {ellapsed_time:.01f} min')
        

## Group the ages together adaptively to reduce bins with small counts

In [ ]:
# Get concept age-group distributions
cagds = dict()
concept_counts = dict()
low_count = 9
low_count_marker = 1
t1 = time()

for i, (c, age_counter) in enumerate(cads.items()):    
    # Get the total concept count, perturbed
    concept_counts[c] = cohd.poisson_perturbation(np.sum(age_counter.counts), n_samples=9)
    
    # Get the grouped age distribution
    cagd = cohd.AdaptiveGroupedAgeCount(age_counter, low_count=low_count, lost_percent_limit=0.02, max_age=90)
    
    # AdaptiveGroupedAgeCount returns None if a suitable grouping wasn't found
    if cagd is not None:
        # Keep track of which bins to suppress
        low_count_bins = (cagd.counts <= low_count) & (cagd.counts > 0)
        
        # Perturbation on all bins
        cagd.counts = cohd.poisson_perturbation(cagd.counts, n_samples=9)

        # Replace low-count bins with suppression marker
        cagd.counts[low_count_bins] = low_count_marker

        cagds[c] = cagd
        
    if i % 10000 == 0:
        ellapsed_time = (time() - t1) / 60
        print(f'{i}: {ellapsed_time}min')
        
# Save the concept age distributions        
with open(path.join(dir_data, 'concept_age_group_dists_perturbed.pkl'), 'wb') as f:
    pickle.dump(cagds, f)
    
# Save the concept counts
with open(path.join(dir_data, 'concept_counts_perturbed.pkl'), 'wb') as f:
    pickle.dump(concept_counts, f)
    
ellapsed_time = (time() - t1) / 60
print(f'{i}: {ellapsed_time}min')            

## Calculate time deltas

In [ ]:
class DeltaCounter():
    """Class for keeping counts where the bins grow as power of 2
    
    For example, with n=5, the bins will be
    <=-16 | -15 — -8 | -7 — -4 | -3 — -2 | -1 | 0 | 1 | 2 — 3 | 4 — 7 | 8 — 15 | >= 16
    """
    # Maximum supported n
    _max_n = 14
    
    # Lookup array for quickly finding out the index offset into the counts list    
    _offset = [0] + [(int(np.log2(x)) + 1) for x in range(1, 2**(_max_n - 1))]    
    
    def __init__(self, n=13):
        assert n >= 0
        self.n = min(n, DeltaCounter._max_n)
        self.bins = self.n * 2 + 1  # Number of bins
        
        # The delta at the start of the last bin. delta >= max_delta all go into the last bin
        self.max_delta = 2**(self.n - 1)  
        
        # counts will be indexed from -n:1:n
        self.counts = [0] * self.bins
        
    def add(self, delta):
        if delta == 0:
            self.counts[self.n] += 1
            return
        
        # Determine the index into counts from the delta
        if delta < 0:
            if delta <= -self.max_delta:
                index = 0
            else:
                index = self.n - min(DeltaCounter._offset[-delta], self.n)
        else:
            if delta >= self.max_delta:
                index = self.bins - 1
            else:
                index = self.n + min(DeltaCounter._offset[delta], self.n)
                
        self.counts[index] += 1
        
    def bin_labels(self):
        labels = [''] * self.n + ['0'] + [''] * self.n
        
        if self.n > 0:
            labels[0] = f'-{2**(self.n-1)}+'
            labels[self.bins - 1] = f'{2**(self.n-1)}+'
            labels[slice(self.n - 1, self.n + 2, 2)] = ['-1', '1']
        
        for i in range(2, self.n):
            lower = 2**(i-1)
            upper = 2**i - 1
            labels[self.n + i] = f'{lower} — {upper}'
            labels[self.n - i] = f'-{upper} — -{lower}'
            
        return labels
    
    def bin_labels_mixed(self):
        """ Compact labels with mixed units (easier to interpret) """
        labels = ['0d', '1d', '2d', '4d', '8d', '16d', '32d', '64d', '128d', '256d', '1.4y', '2.8y', '5.6y', '11.2y', '22.4y', '44.9y', '89.7y', '179y', '359y']
        labels = ['0d']
        
        for i in range(0, min(9, self.n)):
            label = f'{2**i}d'
            labels.append(label)
            labels.insert(0, '-' + label)
            
        
        for i in range(9, self.n):
            label = f'{2**i/365.25:0.1f}y'
            labels.append(label)
            labels.insert(0, '-' + label)
        
        return labels
    
    def x(self):
        """ X-ticks ranging from -self.n:self.n"""
        return np.arange(-self.n, self.n+1)
    
    def normalized_counts(self):
        """ Normalize the counts relative to the number of days in the bin """
        norm = 2**np.maximum(abs(self.x())-1, 0)
        return self.counts / norm
    

### Delta Calculation

In [ ]:
save_intermediates = False
file_backup = file_deltas + file_backup_suffix

# For keeping track of processing time
t1 = time()

# deltas - holds time deltas between pairs of concepts. 
# Indexed by tuple (concept1, concept2) where c1 < c2
deltas = defaultdict(DeltaCounter)  

count = 0

with open(file_pcs) as fh:
    for line in fh:
        split = line.strip().split('\t')
        
        # person_id is the first entry
        pid = int(split.pop(0))
        
        # process the remaining string into a list of occurrence tuples
        occurrences = [_process_occurrence_str(x) for x in split]
        
        # Don't process concepts whose single concept counts are too low
        _trim_low_occurrences(occurrences, cads)
        
        # Expand the list of occurrences with the first instance of their ancestors
        occurrences = _expand_occurrence_ancestors(occurrences, ancestor_map)
        n_occurrences = len(occurrences)
        
        # calculate deltas from each pair of tuples
        for i in range(n_occurrences - 1):
            concept_i = occurrences[i].concept_id
            concept_i_date = occurrences[i].date
            assert concept_i
            
            for j in range(i + 1, n_occurrences):                
                concept_j = occurrences[j].concept_id
                delta = occurrences[j].date - concept_i_date
                
                # don't allow concept_id 0, and two concepts should not be the same
                assert concept_j and concept_i != concept_j
                
                # place the smaller concept_id as the first concept in the tuple
                if concept_i < concept_j:
                    deltas[(concept_i, concept_j)].add(delta.days)
                else:
                    deltas[(concept_j, concept_i)].add(-delta.days)
        
        # display progress
        count += 1
        if count % 10000 == 0:
            ellapsed_time = (time() - t1) / 60
            print(f'{count} - {ellapsed_time:.01f} min')
            
            # Save a backup copy of the data
            if save_intermediates and count % 500000 == 0:
                with open(file_backup, 'wb') as f:
                    pickle.dump(deltas, f)       

# Delete deltas with total co-occurrence <= 10
pairs_to_remove = list()
for concept_pair, delta in deltas.items():
    total_pair_count = np.sum(delta.counts)
    if (total_pair_count) <= 10:
        # Can't delete from dict while iterating, so keep track of keys to delete
        pairs_to_remove.append(concept_pair)
        
# Delete the identified keys and the list of keys
for concept_pair in pairs_to_remove:
    del deltas[concept_pair]
del pairs_to_remove
                    
# Display overall processing time
ellapsed_time = (time() - t1) / 60
print(f'{count} - {ellapsed_time:.01f} min')            
            
# Save the concept age distributions            
with open(file_deltas, 'wb') as f:
    pickle.dump(deltas, f)

# Delete the backup file
if save_intermediates and path.exists(file_backup):
    os.remove(file_backup)            

## Group the delta bins together adaptively to reduce bins with small counts

In [ ]:
t1 = time()
gdcs = dict()
concept_pair_counts = dict()
low_count = 9
low_count_marker = 1
for i, (pair, delta) in enumerate(deltas.items()):
    # Get the total concept pair count, perturbed
    concept_pair_counts[pair] = cohd.poisson_perturbation(np.sum(delta.counts), n_samples=9)
    
    # Get grouped delta counts
    gdc = cohd.AdaptiveGroupedDeltaCount2(delta, settings=[(1, 13), (2, 6), (4, 3), (8, 2), (16, 1)], 
                                          low_count=low_count, lost_percent_limit=0, verbose=False)    
    
    if gdc is not None:
        # Keep track of which bins had low counts
        low_count_bins = (gdc.counts <= low_count) & (gdc.counts > 0)
        
        # Poisson pertubation of all counts
        gdc.counts = cohd.poisson_perturbation(gdc.counts, n_samples=9)
        
        # Replace the low count bins with the value indicating a low count
        gdc.counts[low_count_bins] = low_count_marker
        
        gdcs[pair] = gdc
    
    if i % 100000 == 0:
        ellapsed_time = (time() - t1) / 60
        print(f'{i}: {ellapsed_time} min')
    
# Save the concept age distributions            
with open(path.join(dir_data, 'concept_pair_counts_perturbed.pkl'), 'wb') as f:
    pickle.dump(concept_pair_counts, f)
    
# Save the concept age distributions            
with open(path.join(dir_data, 'deltas_group_perturbed.pkl'), 'wb') as f:
    pickle.dump(gdcs, f)
    
ellapsed_time = (time() - t1) / 60
print(f'ellapsed time: {ellapsed_time} min')